<a href="https://colab.research.google.com/github/Birlinha/dragonAPI/blob/main/DragonAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import json
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dragonAPI = \
    {
    "name":"items",
    "url":"https://ddragon.leagueoflegends.com/cdn/13.23.1/data/en_US/item.json"
    }

In [4]:
def upload_json_to_drive(data, file_name):
    with open(file_name, 'w') as f:
        json.dump(data, f)
    !cp $file_name /content/drive/My\ Drive/
    print(f"{file_name} uploaded to Google Drive.")

def download_json_from_drive(file_name):
    !cp /content/drive/My\ Drive/$file_name $file_name
    with open(file_name, 'r') as f:
        data = json.load(f)
    print(f"{file_name} downloaded.")
    return data


In [5]:
def get_data_from_api(url):
  r = requests.get(url)

  if(r.status_code == 200):
    r_text = r.text
    json_data = json.loads(r_text)

    return json_data

  else:
    print("error on download. Please try manually requesting for data")

In [6]:
print("name: ",dragonAPI["name"])

#data = get_data_from_api(dragonAPI["url"])
#upload_json_to_drive(data, 'items.json')

data = your_json_data = download_json_from_drive('items.json')


name:  items
items.json downloaded.


In [7]:
#print(data)

In [8]:
def create_description(dictionary, depth):
    if depth <= 0 or not isinstance(dictionary, dict):
        return

    depth_folder = "   |--" * depth
    for key in dictionary.keys():
        print(f"{depth_folder}{key}")
        if isinstance(dictionary[key], dict):
            create_description(dictionary[key], depth - 1)


In [9]:
def data_transform():
  items_dict = data["data"]

  df = pd.DataFrame.from_dict(items_dict)

  df_norm = df.T

  df_expand = df_norm.copy()

  df_expand[["map_11","map_12","map_21","map_22","map_30"]] = df_expand.maps.apply(pd.Series)
  df_expand.drop('maps',axis=1, inplace = True)

  df_expand[["gold_base","gold_purchasable","gold_total","gold_self"]] = df_expand.gold.apply(pd.Series)
  df_expand.drop("gold",axis=1, inplace = True)

  df_expand.drop("image",axis=1, inplace = True)

  return df_expand

In [10]:
df_expand = data_transform()

In [11]:
display(df_expand)

,name,description,colloq,plaintext,into,tags,stats,from,depth,inStore,...,requiredChampion,map_11,map_12,map_21,map_22,map_30,gold_base,gold_purchasable,gold_total,gold_self
1001,Boots,<mainText><stats><attention>25</attention> Mov...,;,Slightly increases Move Speed,"[3005, 3158, 3117, 3047, 3020, 3006, 3009, 3111]",[Boots],{'FlatMovementSpeedMod': 25},NaN,NaN,NaN,...,NaN,True,True,True,False,False,300,True,300,210
1004,Faerie Charm,<mainText><stats><attention>50%</attention> Ba...,;,Slightly increases Mana Regen,"[3012, 3114, 4642]",[ManaRegen],{},NaN,NaN,NaN,...,NaN,True,True,True,False,False,250,True,250,175
1006,Rejuvenation Bead,<mainText><stats><attention>100%</attention> B...,;,Slightly increases Health Regen,[3801],[HealthRegen],{},NaN,NaN,NaN,...,NaN,True,True,True,False,False,300,True,300,120
1011,Giant's Belt,<mainText><stats><attention>350</attention> He...,;,Greatly increases Health,"[3075, 3084, 3083, 8001, 3116, 3143, 3748, 463...",[Health],{'FlatHPPoolMod': 350},[1028],2,NaN,...,NaN,True,True,True,False,False,500,True,900,630
1018,Cloak of Agility,<mainText><stats><attention>15%</attention> Cr...,;,Increases critical strike chance,"[6676, 3086, 6672, 3095, 3031, 3033, 3036, 307...",[CriticalStrike],{'FlatCritChanceMod': 0.15},NaN,NaN,NaN,...,NaN,True,True,True,False,False,600,True,600,420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7032,Flicker,<mainText><stats><ornnBonus>85</ornnBonus> Att...,,,NaN,"[Damage, CriticalStrike, CooldownReduction, On...","{'FlatPhysicalDamageMod': 85, 'FlatCritChanceM...",[6675],4,False,...,NaN,True,True,True,False,True,0,False,3400,2380
7033,Cry of the Shrieking City,<mainText><stats><ornnBonus>60</ornnBonus> Abi...,,,NaN,"[Health, SpellDamage, ManaRegen, CooldownReduc...","{'FlatHPPoolMod': 300, 'FlatMagicDamageMod': 60}",[6620],4,False,...,NaN,True,True,True,False,True,0,False,2300,1610
7050,Gangplank Placeholder,,,New Gangplank interface coming soon!,NaN,[],{},NaN,NaN,False,...,Gangplank,True,True,True,False,True,0,False,0,0
8001,Anathema's Chains,<mainText><stats><attention>650</attention> He...,,,NaN,"[Health, Active, CooldownReduction, AbilityHaste]",{'FlatHPPoolMod': 650},"[3067, 1011]",3,NaN,...,NaN,True,True,True,False,False,800,True,2500,1750


In [21]:
from bs4 import BeautifulSoup

def get_stats_bs4(x):

  soup = BeautifulSoup(x, 'html.parser')

  properties = {}
  stats = soup.find('stats')
  if stats:
      for stat in stats.find_all('attention'):
          property_name = stat.find_next_sibling(string=True).strip().lower()
          property_value = stat.text
          properties[property_name] = property_value

  return properties

df_expand["new_description"] = df_expand['description'].apply(lambda x: get_stats_bs4(x))

In [22]:
display(df_expand)

,name,description,colloq,plaintext,into,tags,stats,from,depth,inStore,...,map_11,map_12,map_21,map_22,map_30,gold_base,gold_purchasable,gold_total,gold_self,new_description
1001,Boots,<mainText><stats><attention>25</attention> Mov...,;,Slightly increases Move Speed,"[3005, 3158, 3117, 3047, 3020, 3006, 3009, 3111]",[Boots],{'FlatMovementSpeedMod': 25},NaN,NaN,NaN,...,True,True,True,False,False,300,True,300,210,{'move speed': '25'}
1004,Faerie Charm,<mainText><stats><attention>50%</attention> Ba...,;,Slightly increases Mana Regen,"[3012, 3114, 4642]",[ManaRegen],{},NaN,NaN,NaN,...,True,True,True,False,False,250,True,250,175,{'base mana regen': '50%'}
1006,Rejuvenation Bead,<mainText><stats><attention>100%</attention> B...,;,Slightly increases Health Regen,[3801],[HealthRegen],{},NaN,NaN,NaN,...,True,True,True,False,False,300,True,300,120,{'base health regen': '100%'}
1011,Giant's Belt,<mainText><stats><attention>350</attention> He...,;,Greatly increases Health,"[3075, 3084, 3083, 8001, 3116, 3143, 3748, 463...",[Health],{'FlatHPPoolMod': 350},[1028],2,NaN,...,True,True,True,False,False,500,True,900,630,{'health': '350'}
1018,Cloak of Agility,<mainText><stats><attention>15%</attention> Cr...,;,Increases critical strike chance,"[6676, 3086, 6672, 3095, 3031, 3033, 3036, 307...",[CriticalStrike],{'FlatCritChanceMod': 0.15},NaN,NaN,NaN,...,True,True,True,False,False,600,True,600,420,{'critical strike chance': '15%'}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7032,Flicker,<mainText><stats><ornnBonus>85</ornnBonus> Att...,,,NaN,"[Damage, CriticalStrike, CooldownReduction, On...","{'FlatPhysicalDamageMod': 85, 'FlatCritChanceM...",[6675],4,False,...,True,True,True,False,True,0,False,3400,2380,{'critical strike chance': '20%'}
7033,Cry of the Shrieking City,<mainText><stats><ornnBonus>60</ornnBonus> Abi...,,,NaN,"[Health, SpellDamage, ManaRegen, CooldownReduc...","{'FlatHPPoolMod': 300, 'FlatMagicDamageMod': 60}",[6620],4,False,...,True,True,True,False,True,0,False,2300,1610,{}
7050,Gangplank Placeholder,,,New Gangplank interface coming soon!,NaN,[],{},NaN,NaN,False,...,True,True,True,False,True,0,False,0,0,{}
8001,Anathema's Chains,<mainText><stats><attention>650</attention> He...,,,NaN,"[Health, Active, CooldownReduction, AbilityHaste]",{'FlatHPPoolMod': 650},"[3067, 1011]",3,NaN,...,True,True,True,False,False,800,True,2500,1750,"{'health': '650', 'ability haste': '20'}"


In [23]:
# Function to extract keys from dictionaries
def extract_keys(row):
    return list(row.keys())

# Get unique keys from all dictionaries
unique_keys = set().union(*df_expand['new_description'].apply(extract_keys))

# Create new columns for each unique key and fill with corresponding values
for key in unique_keys:
    df_expand[key] = df_expand['new_description'].apply(lambda x: x.get(key))

# Drop the original column with dictionaries
df_expand.drop(['new_description','description'], axis=1, inplace=True)


In [24]:
display(df_expand)

,name,colloq,plaintext,into,tags,stats,from,depth,inStore,effect,...,magic penetration,base mana regen,base health regen,omnivamp,armor penetration,attack damage,ability haste,armor,health,critical strike damage
1001,Boots,;,Slightly increases Move Speed,"[3005, 3158, 3117, 3047, 3020, 3006, 3009, 3111]",[Boots],{'FlatMovementSpeedMod': 25},NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
1004,Faerie Charm,;,Slightly increases Mana Regen,"[3012, 3114, 4642]",[ManaRegen],{},NaN,NaN,NaN,NaN,...,None,50%,None,None,None,None,None,None,None,None
1006,Rejuvenation Bead,;,Slightly increases Health Regen,[3801],[HealthRegen],{},NaN,NaN,NaN,NaN,...,None,None,100%,None,None,None,None,None,None,None
1011,Giant's Belt,;,Greatly increases Health,"[3075, 3084, 3083, 8001, 3116, 3143, 3748, 463...",[Health],{'FlatHPPoolMod': 350},[1028],2,NaN,NaN,...,None,None,None,None,None,None,None,None,350,None
1018,Cloak of Agility,;,Increases critical strike chance,"[6676, 3086, 6672, 3095, 3031, 3033, 3036, 307...",[CriticalStrike],{'FlatCritChanceMod': 0.15},NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7032,Flicker,,,NaN,"[Damage, CriticalStrike, CooldownReduction, On...","{'FlatPhysicalDamageMod': 85, 'FlatCritChanceM...",[6675],4,False,"{'Effect1Amount': '0.5', 'Effect2Amount': '1',...",...,None,None,None,None,None,None,None,None,None,None
7033,Cry of the Shrieking City,,,NaN,"[Health, SpellDamage, ManaRegen, CooldownReduc...","{'FlatHPPoolMod': 300, 'FlatMagicDamageMod': 60}",[6620],4,False,NaN,...,None,None,None,None,None,None,None,None,None,None
7050,Gangplank Placeholder,,New Gangplank interface coming soon!,NaN,[],{},NaN,NaN,False,NaN,...,None,None,None,None,None,None,None,None,None,None
8001,Anathema's Chains,,,NaN,"[Health, Active, CooldownReduction, AbilityHaste]",{'FlatHPPoolMod': 650},"[3067, 1011]",3,NaN,NaN,...,None,None,None,None,None,None,20,None,650,None
